<a href="https://colab.research.google.com/github/SaguPandya96/Research-Project-Fall-2020/blob/master/Research_Project_Deep_learning_Code_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Research Project DL Code**


**1. Import the libraries And Data**





In [1]:
import pandas as pd
import string

from textblob.classifiers import NaiveBayesClassifier as NBC
from textblob.classifiers import DecisionTreeClassifier
from sklearn.metrics import f1_score
import nltk
nltk.download('punkt')
nltk.download('stopwords')

import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.utils import shuffle
import numpy as np
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**2. Import the Chromium Data**

In [3]:
#load data
DATA_PATH1 = '/content/drive/My Drive/Research Project/chromium.csv'
data1 = pd.read_csv(DATA_PATH1, sep='\t')
data_top1 = data1.head()
print("Chromium Data")
data_top1

Chromium Data


,title,description,type
0,Define a way to enforce performance constraint...,\nThere are cases where tests should fail if a...,Bug
1,Websites that don't work with Chrome Browser,\nChrome Version : 3.0.195.10\r\nURLs (i...,Compat
2,Redraw issue/truncation with confirmation item...,\nChromium 0.3.155.0 (Developer Build 3546)\r\...,Bug
3,Implement CanvasRenderingContext2D for access ...,\nApplications that do background image proces...,Feature
4,ath9k: panic in ath_tx_last_beacon + 0x29a,\nToT (3.0 kernel) panic'd on resume. Was ass...,Bug


**3. Import the Linux Bugs Data**

In [4]:
DATA_PATH2 = '/content/drive/My Drive/Research Project/linux_bugs_usage_ready.csv'
data2 = pd.read_csv(DATA_PATH2, sep='\t')
data_top2 = data2.head()
print("Linux Data")
data_top2

Linux Data


,title,message,importance,product,component
0,oops when using ide-cd with 2.5.45 and cdrecord,Please enter Exact Kernel version:2.5.45 Distr...,P2 normal,IO/Storage,IDE
1,NUMA-Q hangs during TSC initialization on boot.,Exact Kernel version: 2.5.46 Distribution: deb...,P2 normal,Platform Specific/Hardware,i386
2,Enabling shared pagetables causes KDE to wierd...,Exact Kernel version: 2.5.46-mm1 Distribution:...,P2 normal,Memory Management,Other
3,Dcache spirals out of control on 2.5.43-mm2,Exact Kernel version: 2.5.43-mm2 Distribution:...,P2 blocking,IO/Storage,Other
4,64GB highmem BUG(),Exact Kernel version: 2.5.40 Hardware Environm...,P2 normal,Memory Management,Other


**3. Merge title and message for chromium dataset**

In [5]:

def merge_title_and_message(data, message_col_name='message'):
  data['text'] = data['title'] + ' ' + data[message_col_name]
  data = data.drop(['title'], axis=1)
  data = data.drop([message_col_name], axis=1)
  return data
data1 = merge_title_and_message(data1, message_col_name='description')
data_top1 = data1.head()
print("Chromium Merge Title and Message Data")
data_top1

Chromium Merge Title and Message Data


,type,text
0,Bug,Define a way to enforce performance constraint...
1,Compat,Websites that don't work with Chrome Browser \...
2,Bug,Redraw issue/truncation with confirmation item...
3,Feature,Implement CanvasRenderingContext2D for access ...
4,Bug,ath9k: panic in ath_tx_last_beacon + 0x29a \nT...


**4. Merge title and message for Linux Bugs dataset**

In [6]:
data2 = merge_title_and_message(data2)
data_top2 = data2.head()
print("Linux Merge Title and Message Data")
data_top2

Linux Merge Title and Message Data


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using ide-cd with 2.5.45 and cdrecor...
1,P2 normal,Platform Specific/Hardware,i386,NUMA-Q hangs during TSC initialization on boot...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2.5.43-mm2 Ex...
4,P2 normal,Memory Management,Other,64GB highmem BUG() Exact Kernel version: 2.5.4...


**5. Remove the Strip Punctuation For Both the Data**

In [7]:
def strip_punctuations(data, column_name='text'):
  '''
  Strips punctuations from the end of each token.
  This uses suggestion from https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate
  to accomplish this really fast.
  '''
  translator = str.maketrans('', '', string.punctuation)
  data['text'] = data['text'].map(lambda s : str(s).translate(translator))
  return data

In [8]:
data1 = strip_punctuations(data1)
data_top1 = data1.head()
print("Chromium strip punctuations")
data_top1

Chromium strip punctuations


,type,text
0,Bug,Define a way to enforce performance constraint...
1,Compat,Websites that dont work with Chrome Browser nC...
2,Bug,Redraw issuetruncation with confirmation item ...
3,Feature,Implement CanvasRenderingContext2D for access ...
4,Bug,ath9k panic in athtxlastbeacon 0x29a nToT 30 ...


In [9]:
data2 = strip_punctuations(data2)
data_top2 = data2.head()
print("Linux strip punctuations")
data_top2

Linux strip punctuations


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord P...
1,P2 normal,Platform Specific/Hardware,i386,NUMAQ hangs during TSC initialization on boot ...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2543mm2 Exact...
4,P2 normal,Memory Management,Other,64GB highmem BUG Exact Kernel version 2540 Har...


**6. Remove the linux garbage in Linux Bugs data**

In [10]:
def remove_linux_garbage(data):
  '''
  Linux data contains lots of garbage, e.g. memory addresses - 0000f800
  '''
  def is_garbage(w):
    return len(w) >= 7 and sum(c.isdigit() for c in w) >= 2

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if not is_garbage(w) else ' ', s.split())))
  return data
data = remove_linux_garbage(data2)
data_top2 = data2.head()
print("Remove linux garbage")
data_top2

Remove linux garbage


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord P...
1,P2 normal,Platform Specific/Hardware,i386,NUMAQ hangs during TSC initialization on boot ...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on Exact Kerne...
4,P2 normal,Memory Management,Other,64GB highmem BUG Exact Kernel version 2540 Har...


**7. Convert string into lower case with the chromium dataset**

In [11]:
def cast_to_lowercase(data):
  data['text'] = data['text'].map(lambda s : s.lower())
  return data
data1 = cast_to_lowercase(data1)
data2 = cast_to_lowercase(data2)
data_top1 = data1.head()
print("Chromium Lowercase")
data_top1

Chromium Lowercase


,type,text
0,Bug,define a way to enforce performance constraint...
1,Compat,websites that dont work with chrome browser nc...
2,Bug,redraw issuetruncation with confirmation item ...
3,Feature,implement canvasrenderingcontext2d for access ...
4,Bug,ath9k panic in athtxlastbeacon 0x29a ntot 30 ...


**8. Convert string into lower case with the Linux dataset**

In [12]:
data_top2 = data2.head()
print("Linux Lowercase")
data_top2

Linux Lowercase


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord p...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs during tsc initialization on boot ...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde to wierd...
3,P2 blocking,IO/Storage,Other,dcache spirals out of control on exact kerne...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


**9. Remove the stopwords in Chromium dataset**

In [13]:
def remove_stopwords(data):
  stop_words = stopwords.words('english')
  translator = str.maketrans('', '', string.punctuation)
  stop_words = set([w.translate(translator) for w in stop_words]) # Apostrophes were removed already

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if w not in stop_words else ' ', s.split())))
  return data
data1 = remove_stopwords(data1)
data2 = remove_stopwords(data2)
data_top1 = data1.head()
print("Chromium Stopword")
data_top1

Chromium Stopword


,type,text
0,Bug,define way enforce performance constraint ...
1,Compat,websites work chrome browser nchrome ver...
2,Bug,redraw issuetruncation confirmation item d...
3,Feature,implement canvasrenderingcontext2d access ...
4,Bug,ath9k panic athtxlastbeacon 0x29a ntot 30 ke...


**10. Remove the stopwords in Linux dataset**

In [14]:
data_top2 = data2.head()
print("Linux Stopword")
data_top2

Linux Stopword


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please en...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact ...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ...
3,P2 blocking,IO/Storage,Other,dcache spirals control exact kernel vers...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


**11. Remove the rare words in Chromium Dataset**

In [15]:
def remove_rare_words(data, min_count=3):
  wc = {} # WordCount
  def proc_word(s):
    for w in set(s.split()):
      if w in wc:
        wc[w] += 1
      else:
        wc[w] = 1

  for index, row in data.iterrows():
    proc_word(row['text'])

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if wc[w] >= min_count else ' ', s.split())))
  return data
data1 = remove_rare_words(data1)
data2 = remove_rare_words(data2)
data_top1 = data1.head()
print("Chromium rare words")
data_top1

Chromium rare words


,type,text
0,Bug,define way enforce performance constraint spec...
1,Compat,websites work chrome browser nchrome version ...
2,Bug,redraw confirmation item download bar nchrom...
3,Feature,implement canvasrenderingcontext2d access work...
4,Bug,ath9k panic ntot 30 kernel resume associ...


**12. Remove the rare words in Linux Dataset**

In [16]:
data_top2 = data2.head()
print("Linux rare words")
data_top2

Linux rare words


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please enter ex...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact kern...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ex...
3,P2 blocking,IO/Storage,Other,dcache control exact kernel version distribu...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


**13. Import the deep triage library**

In [17]:
# DeepTriage
# This code is adapted from http://bugtriage.mybluemix.net/

import itertools
import numpy as np
np.random.seed(1337)
from gensim.models import Word2Vec
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, merge, BatchNormalization, Bidirectional
from keras.utils import np_utils
from keras.optimizers import RMSprop
from sklearn.utils import shuffle

# Word2vec parameters
min_word_frequency_word2vec = 3
embed_size_word2vec = 200
context_window_word2vec = 5

# Classifier hyperparameters
max_sentence_len = 100 
min_sentence_length = 2
batch_size = 32

In [24]:
def bidir_rnn_classify(data):
  '''
  Use the approach suggested at http://bugtriage.mybluemix.net/ to do
  classification.
  '''
  class_to_predict = 'importance' # component product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]
  unique_train_label = list(set(train_labels))

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  # Tokenize data
  train_data = [text.split() for text in train_data] # TODO(Vladimir) - try nltk tokenize here
  test_data = [text.split() for text in test_data]
  all_data = train_data + test_data

  print('Data examples')
  print(all_data[:5])

  # Generate word2vec
  wordvec_model = Word2Vec(all_data, min_count=min_word_frequency_word2vec,
                           size=embed_size_word2vec, window=context_window_word2vec)
  vocabulary = wordvec_model.wv.vocab
  vocab_size = len(vocabulary)

  print('Vocab size is ' + str(vocab_size))

  X_train = np.empty(shape=[len(train_data), max_sentence_len, embed_size_word2vec],
                     dtype='float32')
  Y_train = np.empty(shape=[len(train_labels), 1], dtype='int32')
  # 1 - start of sentence, # 2 - end of sentence, # 0 - zero padding. Hence, word indices start with 3
  print('Building X_train!')
  for j, curr_row in enumerate(train_data):
    if j % 100 == 0:
      print('Building X_train j = ' + str(j))
    sequence_cnt = 0
    for item in curr_row:
      if item in vocabulary:
        X_train[j, sequence_cnt, :] = wordvec_model[item]
        sequence_cnt = sequence_cnt + 1
        if sequence_cnt == max_sentence_len - 1:
          break
    for k in range(sequence_cnt, max_sentence_len):
      X_train[j, k, :] = np.zeros((1, embed_size_word2vec))
    Y_train[j, 0] = unique_train_label.index(train_labels[j])

  X_test = np.empty(shape=[len(test_data), max_sentence_len, embed_size_word2vec],
                    dtype='float32')
  Y_test = np.empty(shape=[len(test_labels), 1], dtype='int32')
  # 1 - start of sentence, # 2 - end of sentence, # 0 - zero padding. Hence, word indices start with 3
  print('Building X_test!')
  for j, curr_row in enumerate(test_data):
    if j % 100 == 0:
      print('Building X_test j = ' + str(j))
    sequence_cnt = 0
    for item in curr_row:
      if item in vocabulary:
        X_test[j, sequence_cnt, :] = wordvec_model[item]
        sequence_cnt = sequence_cnt + 1
        if sequence_cnt == max_sentence_len - 1:
          break
    for k in range(sequence_cnt, max_sentence_len):
      X_test[j, k, :] = np.zeros((1, embed_size_word2vec))
    Y_test[j, 0] = unique_train_label.index(test_labels[j])

  y_train = np_utils.to_categorical(Y_train, len(unique_train_label))

  print('Bulding KERAS models!') 
#import from keras library 
  sequence = Input(shape=(max_sentence_len, embed_size_word2vec), dtype='float32')
  lstm = Bidirectional(LSTM(1024))(sequence)
  after_dp = Dropout(0.5)(lstm)
  output = Dense(len(unique_train_label), activation='softmax')(after_dp)
  model = Model(sequence, output)
  rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08)
  model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])

  # Train the model
  print('Training the model!')
  hist = model.fit(X_train, y_train,batch_size=batch_size, epochs=5)

  train_result = hist.history
  print(train_result)

  train_prediction = model.predict(X_train)

  total_train_correct = 0
  for j, ll in enumerate(train_prediction):
    if np.argmax(ll) == Y_train[j]:
      total_train_correct += 1

  print('Train accuracy:', total_train_correct * 1.0 / len(train_prediction))

  test_prediction = model.predict(X_test)

  total_test_correct = 0
  labels = []
  predicted = []
  for j, ll in enumerate(test_prediction):
    predicted.append(np.argmax(ll))
    labels.append(Y_test[j])
    if np.argmax(ll) == Y_test[j]:
      total_test_correct += 1

  print('Test accuracy:', total_test_correct * 1.0 / len(test_prediction))
  print('Test F1:', f1_score(labels, predicted, average='weighted'))

  return total_test_correct * 1.0 / len(test_prediction)

In [25]:
bidir_rnn_classify(data2)

Data examples
[['battery', 'charging', 'state', 'change', 'discharging', 'distribution', 'debian', 'unstable', 'hardware', 'environment', 'fujitsu', 'laptop', 'software', 'environment', 'commandline', 'problem', 'description', 'ac', 'power', 'removed', 'reattached', 'update', 'expected', 'charging', 'state', 'listed', 'discharging', 'despite', 'fact', 'laptop', 'plugged', 'strangely', 'enough', 'procacpiacadapteracstate', 'update', 'properly', 'problem', 'also', 'mentioned', 'someone', 'else', 'mailing', 'list', 'steps', 'reproduce', 'cat', 'procacpiacadapteracstate', 'state', 'online', 'cat', 'present', 'yes', 'capacity', 'state', 'ok', 'charging', 'state', 'discharging', 'present', 'rate', '0', 'remaining', 'capacity', '4360', 'mah', 'present', 'voltage', 'mv'], ['merlin', 'pcmcia', 'card', 'recognized', 'serial', 'port', 'recent', 'kernel', 'bug', 'occur', '24x', 'distribution', 'debian', 'hardware', 'environment', 'ibm', 'laptop', 't42', 'software', 'environment', 'kernelorg', 'ker

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Building X_train j = 500
Building X_train j = 600
Building X_train j = 700
Building X_train j = 800
Building X_train j = 900
Building X_train j = 1000
Building X_train j = 1100
Building X_train j = 1200
Building X_train j = 1300
Building X_train j = 1400
Building X_train j = 1500
Building X_train j = 1600
Building X_train j = 1700
Building X_train j = 1800
Building X_train j = 1900
Building X_train j = 2000
Building X_train j = 2100
Building X_train j = 2200
Building X_train j = 2300
Building X_train j = 2400
Building X_train j = 2500
Building X_train j = 2600
Building X_train j = 2700
Building X_train j = 2800
Building X_train j = 2900
Building X_train j = 3000
Building X_train j = 3100
Building X_train j = 3200
Building X_train j = 3300
Building X_train j = 3400
Building X_train j = 3500
Building X_train j = 3600
Building X_train j = 3700
Building X_train j = 3800
Building X_train j = 3900
Building X_train j = 4000
Building X_train j = 4100
Building X_train j = 4200
Building X_train 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Building X_test j = 500
Building X_test j = 600
Building X_test j = 700
Building X_test j = 800
Building X_test j = 900
Building X_test j = 1000
Building X_test j = 1100
Building X_test j = 1200
Building X_test j = 1300
Building X_test j = 1400
Building X_test j = 1500
Building X_test j = 1600
Building X_test j = 1700
Building X_test j = 1800
Building X_test j = 1900
Building X_test j = 2000
Building X_test j = 2100
Building X_test j = 2200
Building X_test j = 2300
Building X_test j = 2400
Bulding KERAS models!
Training the model!
Epoch 1/5
438/438 [==============================] - 4446s 10s/step - loss: 1.3021 - accuracy: 0.5767
Epoch 2/5
438/438 [==============================] - 4143s 9s/step - loss: 1.1484 - accuracy: 0.6136
Epoch 3/5
438/438 [==============================] - 4448s 10s/step - loss: 1.0898 - accuracy: 0.6253
Epoch 4/5
438/438 [==============================] - 4411s 10s/step - loss: 1.0173 - accuracy: 0.6428
Epoch 5/5
438/438 [==============================] - 434

0.6034831915755366

In [18]:
#chromium Data
def bidir_rnn_classify(data):
  '''
  Use the approach suggested at http://bugtriage.mybluemix.net/ to do
  classification.
  '''
  class_to_predict = 'type' # component product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]
  unique_train_label = list(set(train_labels))

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  # Tokenize data
  train_data = [text.split() for text in train_data] # TODO(Vladimir) - try nltk tokenize here
  test_data = [text.split() for text in test_data]
  all_data = train_data + test_data

  print('Data examples')
  print(all_data[:5])

  # Generate word2vec
  wordvec_model = Word2Vec(all_data, min_count=min_word_frequency_word2vec,
                           size=embed_size_word2vec, window=context_window_word2vec)
  vocabulary = wordvec_model.wv.vocab
  vocab_size = len(vocabulary)

  print('Vocab size is ' + str(vocab_size))

  X_train = np.empty(shape=[len(train_data), max_sentence_len, embed_size_word2vec],
                     dtype='float32')
  Y_train = np.empty(shape=[len(train_labels), 1], dtype='int32')
  # 1 - start of sentence, # 2 - end of sentence, # 0 - zero padding. Hence, word indices start with 3
  print('Building X_train!')
  for j, curr_row in enumerate(train_data):
    if j % 100 == 0:
      print('Building X_train j = ' + str(j))
    sequence_cnt = 0
    for item in curr_row:
      if item in vocabulary:
        X_train[j, sequence_cnt, :] = wordvec_model[item]
        sequence_cnt = sequence_cnt + 1
        if sequence_cnt == max_sentence_len - 1:
          break
    for k in range(sequence_cnt, max_sentence_len):
      X_train[j, k, :] = np.zeros((1, embed_size_word2vec))
    Y_train[j, 0] = unique_train_label.index(train_labels[j])

  X_test = np.empty(shape=[len(test_data), max_sentence_len, embed_size_word2vec],
                    dtype='float32')
  Y_test = np.empty(shape=[len(test_labels), 1], dtype='int32')
  # 1 - start of sentence, # 2 - end of sentence, # 0 - zero padding. Hence, word indices start with 3
  print('Building X_test!')
  for j, curr_row in enumerate(test_data):
    if j % 100 == 0:
      print('Building X_test j = ' + str(j))
    sequence_cnt = 0
    for item in curr_row:
      if item in vocabulary:
        X_test[j, sequence_cnt, :] = wordvec_model[item]
        sequence_cnt = sequence_cnt + 1
        if sequence_cnt == max_sentence_len - 1:
          break
    for k in range(sequence_cnt, max_sentence_len):
      X_test[j, k, :] = np.zeros((1, embed_size_word2vec))
    Y_test[j, 0] = unique_train_label.index(test_labels[j])

  y_train = np_utils.to_categorical(Y_train, len(unique_train_label))
  print('Bulding KERAS models!')

  sequence = Input(shape=(max_sentence_len, embed_size_word2vec), dtype='float32')
  lstm = Bidirectional(LSTM(1024))(sequence)
  after_dp = Dropout(0.5)(lstm)
  output = Dense(len(unique_train_label), activation='softmax')(after_dp)
  model = Model(sequence, output)
  rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08)
  model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])

  # Train the model
  print('Training the model!')
  hist = model.fit(X_train, y_train,batch_size=batch_size, epochs=5)

  train_result = hist.history
  print(train_result)

  train_prediction = model.predict(X_train)

  total_train_correct = 0
  for j, ll in enumerate(train_prediction):
    if np.argmax(ll) == Y_train[j]:
      total_train_correct += 1

  print('Train accuracy:', total_train_correct * 1.0 / len(train_prediction))

  test_prediction = model.predict(X_test)

  total_test_correct = 0
  labels = []
  predicted = []
  for j, ll in enumerate(test_prediction):
    predicted.append(np.argmax(ll))
    labels.append(Y_test[j])
    if np.argmax(ll) == Y_test[j]:
      total_test_correct += 1

  print('Test accuracy:', total_test_correct * 1.0 / len(test_prediction))
  print('Test F1:', f1_score(labels, predicted, average='weighted'))

  return total_test_correct * 1.0 / len(test_prediction)

In [ ]:
bidir_rnn_classify(data1)

Data examples
[['cannot', 'upload', 'photos', 'facebook', 'nchrome', 'version', '9059719rnurl', 'httpwwwfacebookcomrnbehavior', 'safari', '4x5xnbehavior', 'firefox', '3x4xnrnwhat', 'steps', 'reproduce', 'problemn1', 'using', 'chrome', 'upload', 'facebook', 'long', 'wait', 'one', 'upload', 'done', 'difference', 'create', 'new', 'album', 'add', 'new', 'photos', 'existing', 'try', 'firefox', 'browser', 'action', 'uploading', 'photos', 'facebook', 'bug', 'uploading', 'process', 'done', 'meet', 'problem', 'using', 'previous', 'chrome', 'version', 'week', 'situation'], ['legacy', 'webaudio', 'remove', 'nthis', 'tracking', 'bug', 'issues', 'legacy', 'webaudio', 'methods', 'attributes', 'need', 'removed', 'support', 'prefixed', 'audiocontext', 'removedn'], ['gn', 'need', 'mechanism', 'share', 'parts', 'gn', 'config', 'nv8', 'rolls', 'srcbuild', 'dependency', 'gn', 'time', 'eg', 'somebody', 'adds', 'something', 'build', 'requires', 'whitelist', 'v8s', 'gn', 'run', 'breaks', 'whitelisting', 'add

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Building X_train j = 600
Building X_train j = 700
Building X_train j = 800
Building X_train j = 900
Building X_train j = 1000
Building X_train j = 1100
Building X_train j = 1200
Building X_train j = 1300
Building X_train j = 1400
Building X_train j = 1500
Building X_train j = 1600
Building X_train j = 1700
Building X_train j = 1800
Building X_train j = 1900
Building X_train j = 2000
Building X_train j = 2100
Building X_train j = 2200
Building X_train j = 2300
Building X_train j = 2400
Building X_train j = 2500
Building X_train j = 2600
Building X_train j = 2700
Building X_train j = 2800
Building X_train j = 2900
Building X_train j = 3000
Building X_train j = 3100
Building X_train j = 3200
Building X_train j = 3300
Building X_train j = 3400
Building X_train j = 3500
Building X_train j = 3600
Building X_train j = 3700
Building X_train j = 3800
Building X_train j = 3900
Building X_train j = 4000
Building X_train j = 4100
Building X_train j = 4200
Building X_train j = 4300
Building X_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Building X_test j = 100
Building X_test j = 200
Building X_test j = 300
Building X_test j = 400
Building X_test j = 500
Building X_test j = 600
Building X_test j = 700
Building X_test j = 800
Building X_test j = 900
Building X_test j = 1000
Building X_test j = 1100
Building X_test j = 1200
Building X_test j = 1300
Building X_test j = 1400
Building X_test j = 1500
Building X_test j = 1600
Building X_test j = 1700
Building X_test j = 1800
Building X_test j = 1900
Building X_test j = 2000
Building X_test j = 2100
Building X_test j = 2200
Building X_test j = 2300
Building X_test j = 2400
Building X_test j = 2500
Building X_test j = 2600
Building X_test j = 2700
Building X_test j = 2800
Building X_test j = 2900
Building X_test j = 3000
Building X_test j = 3100
Building X_test j = 3200
Building X_test j = 3300
Building X_test j = 3400
Building X_test j = 3500
Building X_test j = 3600
Building X_test j = 3700
Building X_test j = 3800
Building X_test j = 3900
Building X_test j = 4000
Building 

In [ ]:
#Hirarchical Attention

import re
from itertools import chain
import string
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from keras import backend as K
from keras.layers import Dense, Input
from keras.layers import GRU, Bidirectional, TimeDistributed, CuDNNLSTM, LSTM, Dropout, CuDNNGRU
from keras.models import Model
from keras.optimizers import  RMSprop
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

In [ ]:
from keras import backend as K
from keras import initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
RNN = CuDNNGRU
X, Y = data1

enc = LabelEncoder()
yc = enc.fit_transform(Y)
oh = LabelBinarizer()
y_trans = oh.fit_transform(yc)
translator = str.maketrans('', '', string.punctuation)
stop_words = stopwords.words('english')
stop_words = set([w.translate(translator) for w in stop_words])

In [ ]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    string = string.strip().lower().translate(translator)
    return string

def remove_stopwords_from_sent(sent):
    res = []
    for word in sent:
        if word not in stop_words:
            res.append(word)
    return res


def build_sentences(X):
    X_sentences = []
    for doc in X:
        sentences = sent_tokenize(doc)
        cleaned = map(clean_str, sentences)
        tokenized = map(word_tokenize, cleaned)
        cleaned = map(remove_stopwords_from_sent, tokenized)
        X_sentences.append(list(cleaned))

    return X_sentences

X_sentences = build_sentences(X)

list(map(print, X_sentences[2]))

In [ ]:
# Word2vec parameters
min_word_frequency_word2vec = 3
embed_size_word2vec = 200
context_window_word2vec = 5

X_merged = list(map(lambda l: list(chain(*l)), X_sentences))

print(X_merged[13])

In [ ]:
wordvec_model = Word2Vec(X_merged, min_count=min_word_frequency_word2vec,
                         size=embed_size_word2vec, window=context_window_word2vec)
max_doc_len = 5
max_sentence_len = 100
num = len(X_sentences)

vocabulary = wordvec_model.wv.vocab
print("Vocabulary", len(vocabulary))

In [ ]:
def map_sentence(sent):
    out = np.empty((max_sentence_len, embed_size_word2vec))
    for ind, word in enumerate(sent):
        if ind == max_sentence_len:
            break
        if word in vocabulary:
            out[ind, :] = wordvec_model.wv[word]
    return out


def map_doc(doc):
    out = np.empty((max_doc_len, max_sentence_len, embed_size_word2vec))
    for ind, sent in enumerate(doc):
        if ind == max_doc_len:
            break
        out[ind, :] = map_sentence(sent)
    return out


x = np.empty((num, max_doc_len, max_sentence_len, embed_size_word2vec))

for ind, doc in enumerate(X_sentences):
    x[ind, :] = map_doc(doc)

In [ ]:
def make_model(rnn_dim=64, dense_dim=50):

    def attention_block():
        def f(input):
            rnn = Bidirectional(RNN(rnn_dim, return_sequences=True))(input)
            drop1 = Dropout(0.75)(rnn)
            dense = TimeDistributed(Dense(dense_dim))(drop1)
            drop2 = Dropout(0.6)(dense)
            att = AttentionWithContext()(drop2)
            return att
        return f

    with K.name_scope('sentence_enc'):
        sentence_input = Input(shape=(max_sentence_len, embed_size_word2vec))
        word_att = attention_block()(sentence_input)
        sentEncoder = Model(sentence_input, word_att)

    with K.name_scope('doc_enc'):
        doc_input = Input(shape=(max_doc_len, max_sentence_len, embed_size_word2vec))
        sent_enc = TimeDistributed(sentEncoder)(doc_input)
        doc_att = attention_block()(sent_enc)
        preds = Dense(y_trans.shape[-1], activation='softmax')(doc_att)

        model = Model(doc_input, preds)
        return model

In [ ]:
model = make_model(rnn_dim=64, dense_dim=64)
model.summary()

sz = len(y_trans)
x_train = x[int(0.1 * sz):int(0.95 * sz)]
x_test = np.stack(x[:int(0.1 * sz)], x[int(0.95 * sz):])

y_train = y_trans[int(0.1 * sz):int(0.95 * sz)]
y_test = np.stack(y_trans[:int(0.1 * sz)], y_trans[int(0.95 * sz):])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test, y_test),
          epochs=10, batch_size=16)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def report(x, y):
    labels = np.argmax(y, axis=-1)
    predicted = np.argmax(model.predict(x), axis=-1)

    print("Acc", accuracy_score(labels, predicted))
    print("F1", f1_score(labels, predicted, average='weighted'))

In [ ]:
print("Training")
report(x_train, y_train)
print("Testing")
report(x_test, y_test)